In [2]:
import pandas as pd 
new_2021 = pd.read_csv('202101-202106参数new.csv',encoding='gb18030')
new_2023 = pd.read_csv('202301-202303参数new.csv',encoding='gb18030')
full_data = new_2021.append(new_2023)
full_data.head()

/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_28507/410572704.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_data = new_2021.append(new_2023)


,零件名称,零件号,仓库,上线月份,1,2,3,4,5,6,ABC需求频次分类,ABC价格分类,ABC需求数量分类,'计划员',组装包,被替代件,替代件
0,车内照明灯(珍珠灰色),6Q0947105F Y20,1000-2,202106,1,NaN,NaN,NaN,NaN,NaN,F,B,SLOW,032,NaN,被替代件,NaN
1,弹性带夹箍,N 90687205,1000-1,202103,2,NaN,4.0,NaN,NaN,NaN,B,D,FAST,001,NaN,NaN,替代件
2,支架,3V5804299,1000-2,202101,4,4.0,8.0,14.0,13.0,11.0,D,D,MIDDLE,003,NaN,NaN,NaN
3,标志牌,33D010467N,1000-1,202103,1,NaN,NaN,NaN,NaN,NaN,F,D,SLOW,048,NaN,NaN,NaN
4,蓄电池负极电线束,561971243A,1000-1,202102,1,NaN,NaN,NaN,NaN,NaN,F,C,SLOW,048,NaN,NaN,NaN


In [3]:
full_data.columns=['零件名称','零件号', '仓库', '上线月份', '1', '2', '3', '4', '5', '6', 'ABC需求频次分类',
       'ABC价格分类', 'ABC需求数量分类', '计划员', '组装包', '被替代件', '替代件']

In [4]:
def month_fillna(data):
    for i in ['1','2','3','4','5','6']:
        data[i].fillna(0,inplace =True)
    return data

full_data = month_fillna(full_data)

full_data.head()

,零件名称,零件号,仓库,上线月份,1,2,3,4,5,6,ABC需求频次分类,ABC价格分类,ABC需求数量分类,计划员,组装包,被替代件,替代件
0,车内照明灯(珍珠灰色),6Q0947105F Y20,1000-2,202106,1,0.0,0.0,0.0,0.0,0.0,F,B,SLOW,032,NaN,被替代件,NaN
1,弹性带夹箍,N 90687205,1000-1,202103,2,0.0,4.0,0.0,0.0,0.0,B,D,FAST,001,NaN,NaN,替代件
2,支架,3V5804299,1000-2,202101,4,4.0,8.0,14.0,13.0,11.0,D,D,MIDDLE,003,NaN,NaN,NaN
3,标志牌,33D010467N,1000-1,202103,1,0.0,0.0,0.0,0.0,0.0,F,D,SLOW,048,NaN,NaN,NaN
4,蓄电池负极电线束,561971243A,1000-1,202102,1,0.0,0.0,0.0,0.0,0.0,F,C,SLOW,048,NaN,NaN,NaN


In [5]:
mask = ~(full_data['替代件'] == '替代件') & ~(full_data['组装包'] == '组装包') & ~full_data['计划员'].isin(['004', '005', '007'])  
full_data = full_data[mask]  


In [6]:
#第二个月预测

# pred_datasets=full_data[['零件名称','零件号','仓库','上线月份','ABC需求数量分类','ABC价格分类','1','2','']]

# 定义一个函数来计算新列的值
def calculate_new_column(row):
    if row['1'] <= 3 or row['ABC需求数量分类'] == 'SLOW':
        return 0
    elif row['ABC价格分类'] in ['A', 'C','D']:
        return row['1'] * 0.2
    
    elif row['1'] > 3 and row['1'] <= 10:
        return row['1'] * 0.5
    
    elif row['1'] > 10 and row['1'] <= 50:
        return row['1'] * 0.3
    elif row['1'] > 50 and row['1'] <= 100:
        return row['1'] * 0.2
    else:
        return row['1'] * 0.1

# 使用apply函数将新列添加到DataFrame中
full_data['pred_2'] = full_data.apply(calculate_new_column, axis=1)


In [7]:
# 使用条件表达式创建新列
full_data['pred_3'] = full_data.apply(lambda row: row['1'] * 0.5 if row['1']>row['2'] else row['1'] * 1, axis=1)

# 使用条件表达式创建新列
full_data['pred_4'] = full_data.apply(lambda row: row['2'] * 0.7 if row['2'] > row['3'] else row['2'] * 1.13, axis=1)

# 使用条件表达式创建新列
full_data['pred_5'] = full_data.apply(lambda row: row['3'] * 0.7 if row['3'] > row['4'] else row['3'] * 1.13, axis=1)

# 使用条件表达式创建新列
full_data['pred_6'] = full_data.apply(lambda row: row['4'] * 0.7 if row['4'] > row['5'] else row['4'] * 1.13, axis=1)



In [8]:
high_demods_info = full_data[full_data['1']>=100]
low_demods_info = full_data[full_data['1']<100]
high_demods_info['pred_3'] = 0
high_demods_info['pred_4'] = 0
high_demods_info['pred_5'] = 0
high_demods_info['pred_6'] = 0
full_data = high_demods_info.append(low_demods_info)

/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_28507/2313734114.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_demods_info['pred_3'] = 0
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_28507/2313734114.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_demods_info['pred_4'] = 0
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_28507/2313734114.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [9]:
for i in ['pred_2','pred_3','pred_4','pred_5','pred_6','6','2','3','4','5']:
    full_data[i] = [1.01 if x ==0 else x for x in full_data[i]]
    
full_data['2_mape'] = abs(full_data['2']-full_data['pred_2'])/full_data['2']
full_data['3_mape'] = abs(full_data['3']-full_data['pred_3'])/full_data['3']
full_data['4_mape'] = abs(full_data['4']-full_data['pred_4'])/full_data['4']
full_data['5_mape'] = abs(full_data['5']-full_data['pred_5'])/full_data['5']
full_data['6_mape'] = abs(full_data['6']-full_data['pred_6'])/full_data['6']
full_data[['2_mape','3_mape','4_mape','5_mape','6_mape']].mean()

2_mape    0.346852
3_mape    0.826076
4_mape    0.483956
5_mape    0.853929
6_mape    0.433949
dtype: float64

In [11]:
full_data.to_csv('1111.csv',encoding  ='gb18030')

In [ ]:
# 计算mape，wmape 占比情况

# 定义区间
import matplotlib.pyplot as plt
import seaborn as sns

bins = [0, 0.10, 0.30, 0.50, 1.00, 100000]
test = full_data.copy()
# 使用cut函数将数据分成不同区间
mape_info = pd.DataFrame()
for i in ['2_mape','3_mape','4_mape','5_mape','6_mape']:
    test[i] = pd.cut(test[i], bins=bins, labels=['0-10%', '10%-30%', '30%-50%', '50%-100%', '>100%'],right =False)
    test[i].replace('NaN','0',inplace =True)
# 计算每个区间中的数据数量
    print('新品第',i.split('_')[0],'月误差')
    df = test[i].value_counts(normalize=True).reset_index().sort_values(by=['index'])
    mape_info = pd.concat([mape_info, df.reset_index()], axis=1)  # axis=1 表示合并列  
mape_info

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
def compare_two_col(r,p):
    
    new_df = full_data[[r,p]]
    new_df =new_df[new_df[r]!=1.01]
    new_df['mape'] = abs(new_df[p]-new_df[r])/new_df[r]
    bins = [0, 0.10, 0.30, 0.50, 1.00, 100000]
# 使用cut函数将数据分成不同区间
    new_df['mape_rate'] = pd.cut(new_df['mape'], bins=bins, labels=['0-10%', '10%-30%', '30%-50%', '50%-100%', '>100%'],right =False)
    new_df['mape_rate'].replace('NaN','0',inplace =True)
# 计算每个区间中的数据数量
    print(new_df['mape'].mean())
    print(new_df['mape_rate'].value_counts(normalize=True).reset_index().sort_values(by=['index']))
    
    
print('2月份')
compare_two_col('2','pred_2')
print('3月份')

compare_two_col('3','pred_3')
print('4月份')

compare_two_col('4','pred_4')
print('5月份')

compare_two_col('5','pred_5')
print('6月份')

compare_two_col('6','pred_6')
